In [1]:
!pip install bs4
!pip install lxml

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

First I need to scrape the data from the wiki page. I had some annoying issues with lxml dependency but figured it out.

In [67]:
#Read page, make dataframe
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(website_url.content,'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))[0]

Now I am going to rename the column "Postal Code" -> "PostalCode", and remove those Boroughs which are Not Assigned. 

In [68]:
#Clean data
# df = df.reset_index
df.rename(columns={'Postal code': 'PostalCode'}, inplace=True) # Change name
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True) # Dropping those without boroughs  
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

Now I will group by "PostalCode". This took me a long time to figure out and then in the end it was already done? Maybe they decided it was too difficult. Then I formatted the strings in "Neighborhood" with column separators. Finally I checked if any of the Neighborhood values are Not Assigned but found that none of them are and so I don't need to replace any values.

In [69]:
# Group Neighborhoods by their postal code
df.groupby(['PostalCode'])['Neighborhood'].apply(','.join)
# This was already done for me?

# Replace / with , in "Neighborhood"
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',', regex=True)

# Checking if any "Neighborhood" values are Not Assigned
df.loc[df["Neighborhood"].notnull()] 
# This was already done for me also?

df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [70]:
df.shape

(103, 3)